In [1]:
import pandas as pd
import transformers
from transformers import AutoTokenizer

c:\Users\JAGRIT BHARATI\Desktop\LLM\llmven\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import Dataset
from datasets import load_dataset

In [3]:
translation_dataset = load_dataset('rahular/itihasa')

Using the latest cached version of the dataset since rahular/itihasa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'Itihasa' at C:\Users\JAGRIT BHARATI\.cache\huggingface\datasets\rahular___itihasa\Itihasa\1.0.0\56645be151b61e1143597f922ccf666b43a5c02b (last modified on Wed Jul  3 22:02:42 2024).


In [6]:
translation_dataset['train'][0]

{'translation': {'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.',
  'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}}

In [7]:
checkpoint = 'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
source_language = 'sn'
target_language = 'en'
prefix = "translate Sanskrit to English"

Here we will write the preprocessing function which will preprocess the inputs, the corrosponding targets. 

We will also use these inputs and targets (they are in list form now) which will be used for the tokenizer to tokenize the variables.

In [9]:
def preprocess_text(input_data):
    inputs = []
    targets = []
    for i in input_data["translation"]:
        inputs.append(prefix + i[source_language])
        targets.append(i[target_language])

    model_inputs = tokenizer(inputs,text_target = targets, max_length = 128, padding = True, truncation = True)
    return model_inputs


Using a mapping function to map the dataset to according to our preprocess function

In [10]:
tokenized_data = translation_dataset.map(preprocess_text,batched=True)

In [11]:
from transformers import DataCollatorForSeq2Seq

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model = checkpoint)

In [13]:
import evaluate
metric = evaluate.load('sacrebleu')

Using the latest cached version of the module from C:\Users\JAGRIT BHARATI\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--sacrebleu\28676bf65b4f88b276df566e48e603732d0b4afd237603ebdf92acaacf5be99b (last modified on Sun Jun 30 16:10:58 2024) since it couldn't be found locally at evaluate-metric--sacrebleu, or remotely on the Hugging Face Hub.


In [15]:
import numpy as np

def postprocess_text(preds,labels):
    preds = []
    labels = []
    for pred in preds:
        pred.append(pred.strip())
    for label in labels:
        label.append(label.strip())

    return preds,labels


def compute_metrics(eval_preds):
    preds,labels= eval_preds
    if isinstance(preds,tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds,skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels,skip_special_tokens=True)

    decoded_preds,decoded_labels = postprocess_text(decoded_preds,decoded_labels)

    result =metric.compute(predictions = decoded_preds,references = decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = []
    pad_token_id = tokenizer.pad_token_id
    for pred in preds:
        count_non_padded = np.count_nonzero(pred != pad_token_id)
        prediction_lens.append(count_non_padded)
    
    result["gen_len"] = np.mean(prediction_lens)
    new_result = {}
    for key,value in result.items():
        new_result[key] = round(value,4)
    result = new_result
    return result

In [16]:
from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainer,Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [17]:
training_arguments = Seq2SeqTrainingArguments(
    output_dir = "my_translation_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay = 0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True 
)


In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args = training_arguments,
    train_dataset= tokenized_data["train"],
    eval_dataset= tokenized_data["test"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [19]:
trainer.train()

  1%|          | 500/46980 [24:40<37:00:07,  2.87s/it]

{'loss': 2.2348, 'grad_norm': 0.5131970643997192, 'learning_rate': 1.9787143465304387e-05, 'epoch': 0.11}


  2%|▏         | 1000/46980 [48:31<37:19:23,  2.92s/it]

{'loss': 1.6914, 'grad_norm': 0.44326817989349365, 'learning_rate': 1.957428693060877e-05, 'epoch': 0.21}


  3%|▎         | 1500/46980 [1:13:18<37:16:43,  2.95s/it]

...

 99%|█████████▉| 46500/46980 [45:19:51<26:55,  3.37s/it]

{'loss': 1.3643, 'grad_norm': 0.3743859529495239, 'learning_rate': 2.0434227330779057e-07, 'epoch': 9.9}


100%|██████████| 46980/46980 [45:47:02<00:00,  2.99s/it]c:\Users\JAGRIT BHARATI\Desktop\LLM\llmven\Lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                        
100%|██████████| 46980/46980 [46:07:31<00:00,  3.53s/it]

{'eval_loss': 1.2871288061141968, 'eval_bleu': 0.3702, 'eval_gen_len': 19.0, 'eval_runtime': 1227.9835, 'eval_samples_per_second': 9.546, 'eval_steps_per_second': 0.597, 'epoch': 10.0}
{'train_runtime': 166051.4706, 'train_samples_per_second': 4.526, 'train_steps_per_second': 0.283, 'train_loss': 1.4298106538229367, 'epoch': 10.0}


TrainOutput(global_step=46980, training_loss=1.4298106538229367, metrics={'train_runtime': 166051.4706, 'train_samples_per_second': 4.526, 'train_steps_per_second': 0.283, 'total_flos': 2.543140120559616e+16, 'train_loss': 1.4298106538229367, 'epoch': 10.0})

In [20]:
model_save_path = "/.llm_translational_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/.llm_translational_model\\tokenizer_config.json',
 '/.llm_translational_model\\special_tokens_map.json',
 '/.llm_translational_model\\spiece.model',
 '/.llm_translational_model\\added_tokens.json',
 '/.llm_translational_model\\tokenizer.json')

In [21]:
example_text = "तस्माच्चिरायुरिच्छत नृपेण मूर्खोऽनुचरो न रक्षणियः ।"
example_text_1 = "अव्यापारेषु व्यापारं यो नरः कर्तुमिच्छति ।"
translated_text = "A king wishing long life should never keep foolish servants."
translated_text_1 = "Anyone who tries to poke into matters which are none of his business,"


In [27]:
from transformers import pipeline
translator = pipeline("translation_en_to_sn", model = model_save_path)
translator(example_text)

[{'translation_text': 'O king, I have been able to slay you, O king,'}]

In [23]:
example_tokenizer = AutoTokenizer.from_pretrained(model_save_path)
inputs = tokenizer(example_text_1,return_tensors="pt").input_ids

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_save_path)
outputs = model.generate(inputs,max_new_tokens = 48, do_sample= True,top_k = 30, top_p = 8.95)

In [28]:
tokenizer.decode(outputs[0],skip_special_tokens=True)

'In that battle the king of women and the kings, that man of great oblation, on each other, has come here. In the battle of the enemy, the king of nitrator and'